<a href="https://colab.research.google.com/github/mohamedashraf111999/ML/blob/master/Leaves_deasis_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision
from pathlib import Path

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !mv pytorch-deep-learning/helper_functions.py . # get the helper_functions.py script
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4028, done.
remote: Counting objects: 100% (1216/1216), done.
remote: Compressing objects: 100% (216/216), done.
remote: Total 4028 (delta 1065), reused 1095 (delta 997), pack-reused 2812
Receiving objects: 100% (4028/4028), 651.38 MiB | 15.84 MiB/s, done.
Resolving deltas: 100% (2358/2358), done.
Updating files: 100% (248/248), done.


In [ ]:
pip install split-folders

In [ ]:
import os
import zipfile

data_path = Path("data/")
tar_file = "PlantVillage_30%.zip"
destination = "PlantVillage"
image_path = data_path / destination
from pathlib import Path
with zipfile.ZipFile(data_path / tar_file, "r") as zip_ref:
  print(f"[INFO] Unzipping {tar_file} data...")
  zip_ref.extractall(image_path)

[INFO] Unzipping PlantVillage_30%.zip data...


In [ ]:
from google.colab import output
import splitfolders
input_folder = "data/PlantVillage/PlantVillage_30%"
splitfolders.ratio(input_folder, output= "PlantVillage_30%", seed = 42, ratio = (.7, .3, .0), group_prefix= None)


Copying files: 6186 files [00:00, 7184.60 files/s]


In [ ]:
train_dir =  "PlantVillage_30%/train"
test_dir = "PlantVillage_30%/val"

train_dir, test_dir

('PlantVillage_30%/train', 'PlantVillage_30%/val')

In [ ]:
print(class_names)

['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
model = torchvision.models.vit_b_16()
model

In [ ]:
def create_vit_b_16_model(num_classes:int=15,
                          seed:int=42):
    """Creates an EfficientNetB2 feature extractor model and transforms.

    Args:
        num_classes (int, optional): number of classes in the classifier head.
            Defaults to 3.
        seed (int, optional): random seed value. Defaults to 42.

    Returns:
        model (torch.nn.Module): EffNetB2 feature extractor model.
        transforms (torchvision.transforms): EffNetB2 image transforms.
    """
    # 1, 2, 3. Create EffNetB2 pretrained weights, transforms and model
    weights = torchvision.models.ViT_B_16_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.vit_b_16(weights=weights)

    # 4. Freeze all layers in base model
    for param in model.parameters():
        param.requires_grad = False

    # 5. Change classifier head with random seed for reproducibility
    torch.manual_seed(seed)
    model.heads = nn.Sequential(
        nn.Linear(in_features=768, out_features=num_classes, bias=True)
    )

    return model, transforms

In [ ]:
vit_b_16, vit_b_16_transforms = create_vit_b_16_model(num_classes=15, seed = 42)

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:05<00:00, 67.1MB/s]


In [ ]:
vit_b_16

In [ ]:
vit_b_16_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [ ]:
from going_modular.going_modular import data_setup
train_dataloader_vit_b_16, test_dataloader_vit_b_16, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                                 test_dir=test_dir,
                                                                                                 transform=vit_b_16_transforms,
                                                                                                 batch_size=16)

In [ ]:
from going_modular.going_modular import engine

# Setup optimizer
optimizer = torch.optim.Adam(params=vit_b_16.parameters(),
                             lr=0.01)
# Setup loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Set seeds for reproducibility and train the model
set_seeds()
vit_b_16__results = engine.train(model=vit_b_16,
                                train_dataloader=train_dataloader_vit_b_16,
                                test_dataloader=test_dataloader_vit_b_16,
                                epochs=10,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.5878 | train_acc: 0.8263 | test_loss: 0.3901 | test_acc: 0.8723
Epoch: 2 | train_loss: 0.2344 | train_acc: 0.9274 | test_loss: 0.4773 | test_acc: 0.8869
Epoch: 3 | train_loss: 0.1352 | train_acc: 0.9518 | test_loss: 0.3140 | test_acc: 0.9176
Epoch: 4 | train_loss: 0.1344 | train_acc: 0.9608 | test_loss: 0.3717 | test_acc: 0.9106
Epoch: 5 | train_loss: 0.0881 | train_acc: 0.9732 | test_loss: 0.2420 | test_acc: 0.9359
Epoch: 6 | train_loss: 0.0991 | train_acc: 0.9698 | test_loss: 0.3133 | test_acc: 0.9273
Epoch: 7 | train_loss: 0.1221 | train_acc: 0.9670 | test_loss: 0.5265 | test_acc: 0.8906
Epoch: 8 | train_loss: 0.0761 | train_acc: 0.9809 | test_loss: 0.2497 | test_acc: 0.9407
Epoch: 9 | train_loss: 0.0928 | train_acc: 0.9735 | test_loss: 0.3706 | test_acc: 0.9300
Epoch: 10 | train_loss: 0.0702 | train_acc: 0.9795 | test_loss: 0.2943 | test_acc: 0.9434


In [ ]:
vit_b_16.to('cpu')
next(iter(vit_b_16.parameters())).device

device(type='cpu')

In [ ]:
from going_modular.going_modular import utils

utils.save_model(model = vit_b_16, target_dir = 'model', model_name = 'vit_Leaf.pth')


[INFO] Saving model to: model/vit_Leaf.pth


In [ ]:
from typing import Tuple, Dict
from timeit import default_timer as timer
def predict(img):
  s_time = timer()
  img = vit_b_16_transforms(img).unsqueeze(0)
  vit_b_16.eval()
  with torch.inference_mode():
    pred_probs = torch.softmax(vit_b_16(img), dim = 1)

  pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
  pred_time = round(timer() - s_time, 5)


  return pred_labels_and_probs, pred_time

In [ ]:
import random
from PIL import Image
from pathlib import Path

test_data_paths = list(Path(test_dir).glob('*/*'))
print(test_dir)
random_image_path = random.sample(test_data_paths, k=1)[0]
image = Image.open(random_image_path)
print(f"[INFO] Predicting on image at path: {random_image_path}\n")
pred_dict, pred_time = predict(img=image)
print(f"Prediction label and probability dictionary: \n{pred_dict}")
print(f"Prediction time: {pred_time} seconds")

PlantVillage_30%/val
[INFO] Predicting on image at path: PlantVillage_30%/val/Potato___Early_blight/160.JPG

Prediction label and probability dictionary: 
{'Pepper__bell___Bacterial_spot': 1.5267783481931474e-08, 'Pepper__bell___healthy': 2.747859184637491e-07, 'Potato___Early_blight': 0.12206153571605682, 'Potato___Late_blight': 0.877923309803009, 'Potato___healthy': 1.3961116565042175e-06, 'Tomato_Bacterial_spot': 5.577196690587094e-12, 'Tomato_Early_blight': 1.9171826348696186e-09, 'Tomato_Late_blight': 1.2416227036737837e-05, 'Tomato_Leaf_Mold': 6.8406005686305615e-12, 'Tomato_Septoria_leaf_spot': 1.1346909332132782e-06, 'Tomato_Spider_mites_Two_spotted_spider_mite': 3.57691476438049e-09, 'Tomato__Target_Spot': 1.9621634872457605e-10, 'Tomato__Tomato_YellowLeaf__Curl_Virus': 8.203045082750581e-18, 'Tomato__Tomato_mosaic_virus': 3.3356475359624937e-09, 'Tomato_healthy': 1.9589745381409784e-09}
Prediction time: 0.53375 seconds


In [ ]:
max(pred_dict.values())
max_keys = [key for key, value in pred_dict.items() if value == max(pred_dict.values())]
max_keys

['Potato___Late_blight']

In [ ]:
examples_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k = 10)]
examples_list

[['PlantVillage_30%/val/Tomato_Late_blight/9.JPG'],
 ['PlantVillage_30%/val/Pepper__bell___healthy/398.JPG'],
 ['PlantVillage_30%/val/Potato___Early_blight/116.JPG'],
 ['PlantVillage_30%/val/Potato___Late_blight/80.JPG'],
 ['PlantVillage_30%/val/Tomato_healthy/76.JPG'],
 ['PlantVillage_30%/val/Tomato_Bacterial_spot/600.JPG'],
 ['PlantVillage_30%/val/Tomato_Septoria_leaf_spot/352.JPG'],
 ['PlantVillage_30%/val/Tomato_Early_blight/17.JPG'],
 ['PlantVillage_30%/val/Tomato_Late_blight/485.JPG'],
 ['PlantVillage_30%/val/Tomato_Early_blight/252.JPG']]

In [ ]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 11.9 MB/s eta

In [ ]:
from logging import debug
import gradio as gr
title = "Leaf deasis detection"
description = "Plant Leaf deasises detection model using Vit"
article = "Model Created during my study on geological feature extraction model to check the acc, loss for different model parameters."

demo = gr.Interface(fn = predict,
                    inputs = gr.Image(type= 'pil'),
                    outputs = [gr.Label(num_top_classes=15, label = "Predictions"),
                               gr.Number(label = "Prediction time (s)")],
                    examples = examples_list,
                    title = title,
                    description = description,
                    article = article,)
demo.launch(debug = False,
            share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ce78a633b5e6489d0e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
